In [190]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pulp import *

## Reading in Dataset

### Fixed Cost of each WTP

In [191]:
fixed_cost = pd.read_excel("../dataset/fixed_cost.xlsx", index_col=0)
fixed_cost.head()

,Low,High
WTP,,
Ampang Intake,6500,9500
Batang Kali,4980,7270
Bernam River Head,6230,9100
Wangsa Maju,3230,4730
Sungai Tengi,2110,6160


### Capacity of each WTP

In [192]:
capacity = pd.read_excel("../dataset/capacity.xlsx", index_col=0)
capacity.head()

,Capacity
WTP,
Ampang Intake,1000
Batang Kali,1000
Bernam River Head,1000
Wangsa Maju,1000
Sungai Tengi,1000


### Variable Cost for each combination of (DMZ, WTP)

In [193]:
variable_cost = pd.read_excel("../dataset/variable_costs.xlsx", index_col=0)
variable_cost.head()

,Ampang Intake,Batang Kali,Bernam River Head,Wangsa Maju,Sungai Tengi
Variable Costs,,,,,
DMZ1,12,12,12,12,12
DMZ2,13,13,13,13,13
DMZ3,10,10,10,10,10
DMZ4,8,8,8,8,8
DMZ5,5,5,5,5,5


### Transport Cost for each combination of (DMZ, WTP)

In [194]:
transport_cost = pd.read_excel("../dataset/freight_costs.xlsx", index_col=0)
transport_cost.head()

,Ampang Intake,Batang Kali,Bernam River Head,Wangsa Maju,Sungai Tengi
Freight Costs ($/Container),,,,,
DMZ1,0,12250,1100,16100,8778
DMZ2,13335,0,8617,20244,10073
DMZ3,15400,22750,0,43610,14350
DMZ4,16450,22050,28000,0,29750
DMZ5,13650,15400,24500,29400,0


### Demand for each DMZ

In [195]:
demand = pd.read_excel("../dataset/demand.xlsx", index_col=0)
demand.head()

,Demand
DMZ,
DMZ1,2800000
DMZ2,90000
DMZ3,1700000
DMZ4,145000
DMZ5,160000


## Optimisation

In [196]:
# List of all the WTPs
wtp = list(capacity.index)

# List of all the DMZs
dmz = list(demand.index)

# List of (DMZ, WTP) pairs
dmz_wtp_pairs = [(d, w) for d in dmz for w in wtp]

In [197]:
demand.loc[dmz[0], "Demand"]

2800000

In [198]:
# Creating the Linear Optimisation Class
model = LpProblem("Optimising water supply", LpMinimize)

C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [199]:
# Creating Decision Variables
output = LpVariable.dicts("Volume", dmz_wtp_pairs, lowBound=0, upBound=None, cat='continuous')

In [200]:
wtp

['Ampang Intake',
 'Batang Kali',
 'Bernam River Head',
 'Wangsa Maju',
 'Sungai Tengi']

In [201]:
# Define the Objective Function
model += \
     lpSum([fixed_cost.loc[w, "High"] * 1000 for w in wtp]) + \
     lpSum([(variable_cost.loc[d, w] + transport_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])

In [202]:
# Adding Constraints

## Meet demand for each DMZ
for d in dmz:
    model += lpSum([output[(d, w)] for w in wtp]) == demand.loc[d, "Demand"]

## Within the WTP capacity
for w in wtp:
    model += lpSum([output[(d, w)] for d in dmz]) <= capacity.loc[w, "Capacity"] * 1000

## No linkage constraint



In [203]:
# Solve the model
model.solve()

1

In [204]:
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))

Total Costs = 34,586,910,000 ($/Month)

Status: Optimal


## Results

In [205]:
dict_wtp = {}
dict_dmz = {}

In [210]:
# df = pd.DataFrame(0, index=dmz, columns = wtp)
# df

df = pd.DataFrame()
df

""


In [211]:
# Getting the results
for v in model.variables():
    # print(v.name, v.varValue)
    name = v.name.replace("Volume_", "").replace("_", "")
    # print(name)

    combi = eval(name)
    # print(combi[0])

    dmz = combi[0]
    wtp = combi[1]
    volume = v.varValue

    # print("DMZ: ", dmz, " ", "WTP: ", wtp, " ", "Value: ", volume)

    df.loc[dmz, wtp] = volume


In [212]:
df

,AmpangIntake,BatangKali,BernamRiverHead,SungaiTengi,WangsaMaju
DMZ1,1000000.0,910000.0,0.0,140000.0,750000.0
DMZ2,0.0,90000.0,0.0,0.0,0.0
DMZ3,0.0,0.0,1000000.0,700000.0,0.0
DMZ4,0.0,0.0,0.0,0.0,145000.0
DMZ5,0.0,0.0,0.0,160000.0,0.0
